# Import libraries

In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import random
import gc

from pandas import *

# SK-learn libraries.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import re

# scipy
from scipy.signal import butter, lfilter


from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.datasets import make_classification




# Import and Merge The Data

In [160]:
#Remove the channels we don't want 
def Remove_Channels(df):
    df.drop(df.columns[[15,16,20,21,22,25,26,27,28,29,30,31,32]], axis=1, inplace=True)
    return df



path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
train_data_filenames = glob.glob(path + "/subj1_series[1-2]*data.csv") #load only subject 1, series 1-3
list_ = []
Train_Array_Lengths = []
Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_data_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(Remove_Channels(df))
    Train_Array_Lengths.append(len(df))
train_data = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#now with event data
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
train_event_filenames = glob.glob(path + "/subj1_series[1-2]*events.csv") #load only subject 1, series 1-3
list_ = []

Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_event_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
train_events = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#do the same thing and now get your testing data
test_data_filenames = glob.glob(path + "/subj1_series3*data.csv") #load only subject 1, series 1-3
list_ = []
Test_Array_Lengths = []
Start_Time = time.time()
print time.time(), "Merging..."

for file_ in test_data_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(Remove_Channels(df))
    Test_Array_Lengths.append(len(df))
test_data = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#now with event data
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
test_event_filenames = glob.glob(path + "/subj1_series3*events.csv") #load only subject 1, series 1-3
list_ = []

Start_Time = time.time()
print time.time(), "Merging..."

for file_ in test_event_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
test_events = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"

1438832246.06 Merging...
1438832246.06 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series1_data.csv
1438832246.91 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series2_data.csv
Merge Complete: 2.03699994087 total seconds
1438832248.11 Merging...
1438832248.11 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series1_events.csv
1438832248.24 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series2_events.csv
Merge Complete: 0.526999950409 total seconds
1438832248.64 Merging...
1438832248.64 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series3_data.csv
Merge Complete: 1.30500006676 total seconds
1438832249.94 Merging...
1438832249.94 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series3_events.csv
Merge Complete: 0.248999834061 total seconds


# Separate the data into the 6 events and into training and test sets

In [161]:
Train_Labels_HandStart =  train_events['HandStart'].to_sparse(fill_value=0)
Train_Labels_FirstDigitTouch =  train_events['FirstDigitTouch'].to_sparse(fill_value=0)
Train_Labels_BothStartLoadPhase =  train_events['BothStartLoadPhase'].to_sparse(fill_value=0)
Train_Labels_LiftOff =  train_events['LiftOff'].to_sparse(fill_value=0)
Train_Labels_Replace =  train_events['Replace'].to_sparse(fill_value=0)
Train_Labels_BothReleased =  train_events['BothReleased'].to_sparse(fill_value=0)
# Train_Labels_Combined = train_events.HandStart.map(str) + ',' + train_events.FirstDigitTouch.map(str) + ',' + train_events.BothStartLoadPhase.map(str) + ',' + train_events.LiftOff.map(str) + ',' + train_events.Replace.map(str) + ',' + train_events.BothReleased.map(str)


Train_Data = train_data.iloc[0:,1:].astype('float32') # select only column data


Test_Labels_HandStart =  test_events['HandStart'].to_sparse(fill_value=0)
Test_Labels_FirstDigitTouch =  test_events['FirstDigitTouch'].to_sparse(fill_value=0)
Test_Labels_BothStartLoadPhase =  test_events['BothStartLoadPhase'].to_sparse(fill_value=0)
Test_Labels_LiftOff =  test_events['LiftOff'].to_sparse(fill_value=0)
Test_Labels_Replace =  test_events['Replace'].to_sparse(fill_value=0)
Test_Labels_BothReleased =  test_events['BothReleased'].to_sparse(fill_value=0)


Test_Data = test_data.iloc[0:,1:].astype('float32') # select only data columns

#delete what isn't needed anymore to save memory. 
del train_data
del test_data
del train_events
del test_events
gc.collect()

122

# Define functions for feature extraction

In [156]:

#bin the time 
def Bin_Time(num_rows,num_bins):
    Bin_Size = num_rows/num_bins
    Bins = np.zeros(shape=(num_rows,num_bins))
    Bin_Min = 0
    Bin_Max = Bin_Size
    for i in range(0,num_bins):
        Bins[Bin_Min:Bin_Max,i] = 1
        Bin_Min = Bin_Min + Bin_Size
        Bin_Max = Bin_Max + Bin_Size
    return Bins


##filters borrowed from Nihar. 
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


#run PCA and return the number of PCs that explain the given amount of variance. 
def extractFeatures_PCA(Train_Data,Test_Data, PercentVarExplained):
    pca = PCA()
    pca.fit(Train_Data)  
    
    Explained_Variance_Ratios = pca.explained_variance_ratio_
    for i in range(1,len(Explained_Variance_Ratios)):
        if sum(Explained_Variance_Ratios[0:i]) >= PercentVarExplained:
                   NumPCs = i + 1 #add 1 since numpy array ranges are not inclusive
                   break
    PCA_Projections = pca.transform(Train_Data)[:,0:NumPCs]
    PCA_Projections_Test = pca.transform(Test_Data)[:,0:NumPCs]
    return np.float32(PCA_Projections),np.float32(PCA_Projections_Test)



#return rolling mean of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_mean(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_mean(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'mean' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling variance of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_var(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_var(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'var' + str(i)})
    return np.array(new_df.astype('float32'))


#return rolling min of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_min(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_min(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'min' + str(i)})
    return np.array(new_df.astype('float32'))


#return rolling max of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_max(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_max(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'max' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling skew of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_skew(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_skew(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'skew' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling skew of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_kurt(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_kurt(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'kurt' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling quantile of each column in a pandas dataframe with a given window and quantile. Returns df of same size. 
def df_rolling_quantile(df,window,quantile):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_quantile(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'Pct'+ quantile + "_" + str(i)})
    return np.array(new_df.astype('float32'))


#BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
#return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_corr(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(pd.rolling_corr(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)



#BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
#return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_cov(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(rolling_cov(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)




#Feature Creation 

This section takes rolling statistics for different windows of time for each column of the dataframe. It then does an RFE feature selection technique to keep only the top features for each rolling stat for each of the six labels. It then appends all of this together into a customized feature array for each of the six labels.  

This took about 10 minutes on my machine

In [162]:
Windows = [10,100,500,700]

Test_Features = []
Train_Features = []

Start_Time = time.time()
for w in Windows:
    Raw_Data_Rolling_Window = w
    
    
    RawData_Rolling_Means_Train = df_rolling_mean(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Var_Train = df_rolling_var(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Skewness_Train = df_rolling_skew(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Min_Train = df_rolling_min(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Max_Train = df_rolling_max(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)


    RawData_Rolling_Means_Test = df_rolling_mean(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Var_Test = df_rolling_var(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Skewness_Test = df_rolling_skew(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Min_Test = df_rolling_min(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
    RawData_Rolling_Max_Test = df_rolling_max(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)

    print "Calculations With Window of", w, "Complete: ", int(time.time()-Start_Time), " Seconds"

    
    #Append all the features into 1
    Train_Features.append(np.concatenate((RawData_Rolling_Means_Train,RawData_Rolling_Var_Train,RawData_Rolling_Skewness_Train,RawData_Rolling_Min_Train,RawData_Rolling_Max_Train),axis = 1))
    Test_Features.append(np.concatenate((RawData_Rolling_Means_Test,RawData_Rolling_Var_Test,RawData_Rolling_Skewness_Test,RawData_Rolling_Min_Test,RawData_Rolling_Max_Test),axis = 1))
    


# Train_Bin_List = []
# for i in range (0,len(Train_Array_Lengths)):
#     Train_Bin_List.append(Bin_Time(Train_Array_Lengths[i],Initial_Num_Bins)) # bin the time based on the sizes of each series. 
# Train_Time_Bins = np.uint8(np.concatenate(Train_Bin_List, axis = 0))


# Test_Bin_List = []
# for i in range (0,len(Test_Array_Lengths)):
#     Test_Bin_List.append(Bin_Time(Test_Array_Lengths[i],Initial_Num_Bins)) # bin the time based on the sizes of each series. 
# Test_Time_Bins = np.uint8(np.concatenate(Test_Bin_List, axis = 0))

# print "Time Bin Creation Complete: ", int(time.time()-Start_Time), " Seconds"    

# Train_Features.append(Train_Time_Bins) # Add reduced set of features to full set
# Test_Features.append(Test_Time_Bins) # Add reduced set of features to full set    
# print "Time Bins Added to Feature Set: ", int(time.time()-Start_Time), " Seconds"    
# #delete what isn't needed anymore to save memory. 
# del Train_Time_Bins
# del Test_Time_Bins

del RawData_Rolling_Means_Train
del RawData_Rolling_Means_Test

del RawData_Rolling_Var_Train
del RawData_Rolling_Var_Test

del RawData_Rolling_Skewness_Train
del RawData_Rolling_Skewness_Test

del RawData_Rolling_Min_Train
del RawData_Rolling_Min_Test

del RawData_Rolling_Max_Train
del RawData_Rolling_Max_Test
gc.collect()

Test_Features = np.float32(np.concatenate((Test_Features),axis = 1))
gc.collect()

Train_Features =np.float32( np.concatenate((Train_Features),axis = 1))
gc.collect()

Test_Features = np.float32(np.concatenate((Test_Features,Test_Data),axis = 1))
gc.collect()

Train_Features =np.float32( np.concatenate((Train_Features,Train_Data),axis = 1))

del Train_Data
del Test_Data

gc.collect()

print Train_Features.shape[1], "Total Resultant Features"



Calculations With Window of 10 Complete:  14  Seconds
Calculations With Window of 100 Complete:  26  Seconds
Calculations With Window of 500 Complete:  37  Seconds
Calculations With Window of 700 Complete:  48  Seconds
399 Total Resultant Features


## Add additional features here

#Principal Component Analysis

Reduce the dimensionality of our feature set before modeling begins. 

In [163]:
#set the percent of the total variance explained we want explained by our subset of principal components. 
Pct_Variance_Explained = .93


Start_Time = time.time() # begin timer

# we must first scale the data. 
Scale_Center = StandardScaler()
Z_Train_Features = np.float16(Scale_Center.fit_transform(np.array(Train_Features)))
del Train_Features 
gc.collect()


Z_Test_Features = np.float16(Scale_Center.fit_transform(np.array(Test_Features)))
del Test_Features 
gc.collect()


PCs_Train,PCs_Test = extractFeatures_PCA(Z_Train_Features,Z_Test_Features,Pct_Variance_Explained)
del Z_Train_Features,Z_Test_Features 
gc.collect()
print "PCA Complete:", int(time.time()-Start_Time), " Seconds"

print ''
print PCs_Train.shape[1], "Resultant Principal Components"

PCA Complete: 33  Seconds

82 Resultant Principal Components


In [18]:
print PCs_Train.shape

(391450L, 58L)


HandStart_PCs Complete: 15  Seconds
FirstDigitTouch_PCs Complete: 30  Seconds
BothStartLoadPhase_PCs Complete: 44  Seconds
LiftOff_PCs Complete: 60  Seconds
Replace_PCs Complete: 75  Seconds
BothReleased_PCs Complete: 89  Seconds


In [80]:
Pct_Features_To_Keep = .90
Num_Features = PCs_Train.shape[1]

Num_Features_To_Keep = int(Num_Features*Pct_Features_To_Keep)
Num_Steps = 4


C_Value = 1
penalty = 'l1'
Convergence_tol = .01


RFE_Step = int((Num_Features-Num_Features_To_Keep)/Num_Steps)

Start_Time = time.time()



#since the recursive feature elimination uses the coefficients as a means to eliminate features, 
# we must first scale the data. 

Scale_Center = StandardScaler()
Z_Train_Features = np.float16(Scale_Center.fit_transform(np.array(PCs_Train)))
del PCs_Train 
gc.collect()

Z_Test_Features = np.float16(Scale_Center.fit_transform(np.array(PCs_Test)))
del PCs_Test 
gc.collect()

print "Scaling Complete:", int(time.time()-Start_Time), " Seconds"



LogReg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol) 
rfe = RFE(LogReg,Num_Features_To_Keep, step=RFE_Step)

#reduce entire feature set to top x features for HandStart
rfe.fit(Z_Train_Features, Train_Labels_HandStart.to_dense())
HandStart_Train_Features = np.float16(rfe.transform(Z_Train_Features))
gc.collect()

HandStart_Test_Features = np.float16(rfe.transform(Z_Test_Features))
gc.collect()
print "HandStart_Test Features Complete:", int(time.time()-Start_Time), " Seconds"
    
    
#reduce entire feature set to top x features for FirstDigitTouch   
rfe.fit(Z_Train_Features, Train_Labels_FirstDigitTouch.to_dense())
FirstDigitTouch_Train_Features = np.float16(rfe.transform(Z_Train_Features))   
gc.collect()

FirstDigitTouch_Test_Features = np.float16(rfe.transform(Z_Test_Features))
gc.collect()
print "FirstDigitTouch Features Complete:", int(time.time()-Start_Time), " Seconds"



#reduce entire feature set to top x features for BothStartLoadPhase   
rfe.fit(Z_Train_Features, Train_Labels_BothStartLoadPhase.to_dense())
BothStartLoadPhase_Train_Features = np.float16(rfe.transform(Z_Train_Features))   
gc.collect()

BothStartLoadPhase_Test_Features = np.float16(rfe.transform(Z_Test_Features))
gc.collect()
print "BothStartLoadPhase Features Complete:", int(time.time()-Start_Time), " Seconds"



#reduce entire feature set to top x features for LiftOff   
rfe.fit(Z_Train_Features, Train_Labels_LiftOff.to_dense())
LiftOff_Train_Features = np.float16(rfe.transform(Z_Train_Features))   
gc.collect()

LiftOff_Test_Features = np.float16(rfe.transform(Z_Test_Features))
gc.collect()
print "LiftOff Features Complete:", int(time.time()-Start_Time), " Seconds"



#reduce entire feature set to top x features for Replace   
rfe.fit(Z_Train_Features, Train_Labels_Replace.to_dense())
Replace_Train_Features = np.float16(rfe.transform(Z_Train_Features))   
gc.collect()

Replace_Test_Features = np.float16(rfe.transform(Z_Test_Features))
gc.collect()
print "Replace Features Complete:", int(time.time()-Start_Time), " Seconds"



#reduce entire feature set to top x features for BothReleased   
rfe.fit(Z_Train_Features, Train_Labels_BothReleased.to_dense())
BothReleased_Train_Features = np.float16(rfe.transform(Z_Train_Features))   
gc.collect()

BothReleased_Test_Features = np.float16(rfe.transform(Z_Test_Features))
gc.collect()
print "BothReleased Features Complete:", int(time.time()-Start_Time), " Seconds"



#delete what isn't needed anymore to save memory. 
del Z_Train_Features
del Z_Test_Features
gc.collect()


print BothReleased_Test_Features.shape[1], "Features Remaining"

Scaling Complete: 1  Seconds
HandStart_Test Features Complete: 169  Seconds
FirstDigitTouch Features Complete: 237  Seconds
BothStartLoadPhase Features Complete: 342  Seconds
LiftOff Features Complete: 499  Seconds
Replace Features Complete: 539  Seconds
BothReleased Features Complete: 585  Seconds
68 Features Remaining


# Test Different Models

##Logistic Regession

In [121]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#set parameters for Logistic Regression
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
penalty = 'l1'
Convergence_tol = .01

#Create a function that trains and runs a logistic regression model. Then prints and returns the AUC score
def predictCategory_LogReg(train_data, train_label, test_data, test_label, Category, C_Value,penalty, Convergence_tol):
    Logistic_Reg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol)    
    Logistic_Reg.fit(train_data, train_label)
    print(Category, " Score =", Logistic_Reg.score(test_data, test_label))

    prob = Logistic_Reg.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])

Start_Time = time.time()   
AUC_HandStart = predictCategory_LogReg(HandStart_Train_Features,Train_Labels_HandStart.to_dense(),HandStart_Test_Features,Test_Labels_HandStart.to_dense(), 'HandStart', C_Value,penalty, Convergence_tol)
AUC_FirstDigitTouch = predictCategory_LogReg(FirstDigitTouch_Train_Features,Train_Labels_FirstDigitTouch.to_dense(),FirstDigitTouch_Test_Features,Test_Labels_FirstDigitTouch.to_dense(), 'FirstDigitTouch', C_Value,penalty, Convergence_tol)
AUC_BothStartLoadPhase = predictCategory_LogReg(BothStartLoadPhase_Train_Features,Train_Labels_BothStartLoadPhase.to_dense(),BothStartLoadPhase_Test_Features,Test_Labels_BothStartLoadPhase.to_dense(), 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)
AUC_LiftOff = predictCategory_LogReg(LiftOff_Train_Features,Train_Labels_LiftOff.to_dense(),LiftOff_Test_Features,Test_Labels_LiftOff.to_dense(), 'LiftOff', C_Value,penalty, Convergence_tol)
AUC_Replace = predictCategory_LogReg(Replace_Train_Features,Train_Labels_Replace.to_dense(),Replace_Test_Features,Test_Labels_Replace.to_dense(), 'Replace', C_Value,penalty, Convergence_tol)
AUC_BothReleased = predictCategory_LogReg(BothReleased_Train_Features,Train_Labels_BothReleased.to_dense(),BothReleased_Test_Features,Test_Labels_BothReleased.to_dense(), 'BothReleased', C_Value,penalty, Convergence_tol)
print int(time.time()-Start_Time), " Seconds to complete"
print "Overall Logistic Regression Score = ", np.mean((AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased))

('HandStart', ' Score =', 0.97995073846351799)
('HandStart', 'AUC', 0.67537557227244893)
('FirstDigitTouch', ' Score =', 0.98046541123273323)
('FirstDigitTouch', 'AUC', 0.73081823859460271)
('BothStartLoadPhase', ' Score =', 0.98046081594015089)
('BothStartLoadPhase', 'AUC', 0.74570748025188682)
('LiftOff', ' Score =', 0.97743251812842924)
('LiftOff', 'AUC', 0.75471834199300536)
('Replace', ' Score =', 0.97641695846774568)
('Replace', 'AUC', 0.69387548993751025)
('BothReleased', ' Score =', 0.97628829027544184)
('BothReleased', 'AUC', 0.65981917217016595)
102  Seconds to complete
Overall Logistic Regression Score =  0.710052382537


In [164]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#set parameters for Logistic Regression
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
penalty = 'l2'
Convergence_tol = .001

#Create a function that trains and runs a logistic regression model. Then prints and returns the AUC score
def predictCategory_LogReg(train_data, train_label, test_data, test_label, Category, C_Value,penalty, Convergence_tol):
    Logistic_Reg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol)    
    Logistic_Reg.fit(train_data, train_label)
    print(Category, " Score =", Logistic_Reg.score(test_data, test_label))

    prob = Logistic_Reg.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])


Start_Time = time.time()   
AUC_HandStart = predictCategory_LogReg(PCs_Train,Train_Labels_HandStart.to_dense(),PCs_Test,Test_Labels_HandStart.to_dense(), 'HandStart', C_Value,penalty, Convergence_tol)
AUC_FirstDigitTouch = predictCategory_LogReg(PCs_Train,Train_Labels_FirstDigitTouch.to_dense(),PCs_Test,Test_Labels_FirstDigitTouch.to_dense(), 'FirstDigitTouch', C_Value,penalty, Convergence_tol)
AUC_BothStartLoadPhase = predictCategory_LogReg(PCs_Train,Train_Labels_BothStartLoadPhase.to_dense(),PCs_Test,Test_Labels_BothStartLoadPhase.to_dense(), 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)
AUC_LiftOff = predictCategory_LogReg(PCs_Train,Train_Labels_LiftOff.to_dense(),PCs_Test,Test_Labels_LiftOff.to_dense(), 'LiftOff', C_Value,penalty, Convergence_tol)
AUC_Replace = predictCategory_LogReg(PCs_Train,Train_Labels_Replace.to_dense(),PCs_Test,Test_Labels_Replace.to_dense(), 'Replace', C_Value,penalty, Convergence_tol)
AUC_BothReleased = predictCategory_LogReg(PCs_Train,Train_Labels_BothReleased.to_dense(),PCs_Test,Test_Labels_BothReleased.to_dense(), 'BothReleased', C_Value,penalty, Convergence_tol)
print int(time.time()-Start_Time), " Seconds to complete"
print "Overall Logistic Regression Score = ", np.mean((AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased))

('HandStart', ' Score =', 0.96715744391445402)
('HandStart', 'AUC', 0.84271222611360797)
('FirstDigitTouch', ' Score =', 0.97057174630308707)
('FirstDigitTouch', 'AUC', 0.81575167224714584)
('BothStartLoadPhase', ' Score =', 0.96395452498460577)
('BothStartLoadPhase', 'AUC', 0.82450103409558972)
('LiftOff', ' Score =', 0.96218993263301078)
('LiftOff', 'AUC', 0.8379948363275137)
('Replace', ' Score =', 0.97826426608582173)
('Replace', 'AUC', 0.80196850008054998)
('BothReleased', ' Score =', 0.97734980286194817)
('BothReleased', 'AUC', 0.7716553545378152)
118  Seconds to complete
Overall Logistic Regression Score =  0.815763937234


### This is a failed attempt at parallel processing the model 

In [ ]:
import multiprocessing as mp
from multiprocessing import Process
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#set parameters for Logistic Regression
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
penalty = 'l1'
Convergence_tol = .01

#Create a function that trains and runs a logistic regression model. Then prints and returns the AUC score
def predictCategory_LogReg(train_data, train_label, test_data, test_label, Category, C_Value,penalty, Convergence_tol):
    Logistic_Reg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol)    
    Logistic_Reg.fit(train_data, train_label)
    print(Category, " Score =", Logistic_Reg.score(test_data, test_label))

    prob = Logistic_Reg.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])

Start_Time = time.time()
def Log_Reg_HandStart():
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', C_Value,penalty, Convergence_tol)

    
def Log_Reg_FirstDigitTouch():    
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', C_Value,penalty, Convergence_tol)

    
def Log_Reg_BothStartLoadPhase():       
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)

    
def Log_Reg_LiftOff():       
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', C_Value,penalty, Convergence_tol)

    
def Log_Reg_Replace():     
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', C_Value,penalty, Convergence_tol)

    
def Log_Reg_BothReleased():    
    predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', C_Value,penalty, Convergence_tol)

    

def Overall_AUC(i):
    if i == 1:
        Log_Reg_HandStart()
    if i == 2:
        Log_Reg_FirstDigitTouch()
    if i == 3:
        Log_Reg_BothStartLoadPhase()
    if i == 4:
        Log_Reg_LiftOff()
    if i == 5:
        Log_Reg_Replace()
    if i == 6:
        Log_Reg_BothReleased()
    
# print Overall_AUC(AUC_Values)


   
    
pool = mp.Pool(processes=1)
results = [pool.apply_async(Overall_AUC, args=(i,)) for i in range(1,7)]
output = [p.get() for p in results]
print(output)



print time.time()-Start_Time






## SVM Model

This is set up to get started. It has not been tested. MM 08/04/2015 11:30 A.M.

In [ ]:
#set parameters for SVM
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
kernel = 'rbf'
degree = 3

#Create a function that trains and runs a model. Then prints and returns the AUC score
def predictCategory_SVM(train_data, train_label, test_data, test_label, Category, C_Value,kernel, degree):
    SVM = SVC(C = C_Value, kernel = kernel,degree=degree)    
    SVC.fit(train_data, train_label)

    prob = SVC.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])
    
AUC_HandStart = predictCategory_SVM(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', Category, C_Value,kernel, degree)
AUC_FirstDigitTouch = predictCategory_SVM(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', Category, C_Value,kernel, degree)
AUC_BothStartLoadPhase = predictCategory_SVM(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', Category, C_Value,kernel, degree)
AUC_LiftOff = predictCategory_SVM(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', Category, C_Value,kernel, degree)
AUC_Replace = predictCategory_SVM(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', Category, C_Value,kernel, degree)
AUC_BothReleased = predictCategory_SVM(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', Category, C_Value,kernel, degree


print "Overall SVM Score = ", np.mean(AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased)

##Neural Network

In [ ]:
#set parameters for Neural Network model
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
n_components = 10
learning_rate = .1
batch_size = 20000
n_iter =1


#Create a function that trains and runs a model. Then prints and returns the AUC score
def predictCategory_Neural_Network(train_data, train_label, test_data, test_label, Category, C_Value,kernel, degree):
    SVM = SVC(C = C_Value, kernel = kernel,degree=degree)    
    SVC.fit(train_data, train_label)

    prob = SVC.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])
    
AUC_HandStart = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', Category, C_Value,kernel, degree)
AUC_FirstDigitTouch = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', Category, C_Value,kernel, degree)
AUC_BothStartLoadPhase = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', Category, C_Value,kernel, degree)
AUC_LiftOff = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', Category, C_Value,kernel, degree)
AUC_Replace = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', Category, C_Value,kernel, degree)
AUC_BothReleased = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', Category, C_Value,kernel, degree


print "Overall Neural Network Score = ", np.mean(AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased)


## Decision Trees and Random Forest